In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import time
from pandarallel import pandarallel
import math

In [6]:
pandarallel.initialize(progress_bar=True)

def func(x):
    return math.sin(x.a**2) + math.sin(x.b**2)


df_size = int(5e6)
df = pd.DataFrame(dict(a=np.random.randint(1, 8, df_size),
                       b=np.random.rand(df_size)))
print(df.shape)
df.head()

New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 4 workers



(5000000, 2)


,a,b
0,4,0.908441
1,4,0.055522
2,7,0.567725
3,5,0.987823
4,1,0.471693


# DataFrame.apply

In [9]:
start = time.time()
res = df.apply(func, axis=1)
end = time.time()
print('Took %.3f seconds.' % (end - start))


start = time.time()
res_parallel = df.parallel_apply(func, axis=1)
res.equals(res_parallel)
end = time.time()
print('Took %.3f seconds.' % (end - start))

Took 122.655 seconds.
    





Took 44.428 seconds.


# DataFrame.applymap

In [10]:
df_size = int(1e7)
df = pd.DataFrame(dict(a=np.random.randint(1, 8, df_size),b=np.random.rand(df_size)))

def func(x):
    return math.sin(x**2) - math.cos(x**2)

start = time.time()
res = df.applymap(func)
end = time.time()
print('Took %.3f seconds.' % (end - start))

start = time.time()
res_parallel = df.parallel_applymap(func)
res.equals(res_parallel)
end = time.time()
print('Took %.3f seconds.' % (end - start))

Took 17.512 seconds.
    





Took 17.175 seconds.


# DataFrame.groupby.apply

In [11]:
df_size = int(3e7)
df = pd.DataFrame(dict(a=np.random.randint(1, 1000, df_size),
b=np.random.rand(df_size)))
def func(df):
    dum = 0
    for item in df.b:
        dum += math.log10(math.sqrt(math.exp(item**2)))
    return dum / len(df.b)

start = time.time()
res = df.groupby("a").apply(func)
end = time.time()
print('Took %.3f seconds.' % (end - start))


start = time.time()
res_parallel = df.groupby("a").parallel_apply(func)
res.equals(res_parallel)
end = time.time()
print('Took %.3f seconds.' % (end - start))

Took 15.486 seconds.
Took 7.875 seconds.


# DataFrame.groupby.rolling.apply
- 除了df.groupby.col_name.rolling。如果应用程序的速度只增加了x3.2倍，那么平均速度就会增加大约x4倍，即使用的计算机上的内核数量。

In [12]:
df_size = int(1e6)
df = pd.DataFrame(dict(a=np.random.randint(1, 300, df_size),b=np.random.rand(df_size)))

def func(x):
    return x.iloc[0] + x.iloc[1] ** 2 + x.iloc[2] ** 3 + x.iloc[3] ** 4

In [13]:
%%time
res = df.groupby('a').b.rolling(4).apply(func, raw=False)

CPU times: user 1min 37s, sys: 197 ms, total: 1min 37s
Wall time: 1min 37s


In [14]:
%%time
res_parallel = df.groupby('a').b.rolling(4).parallel_apply(func, raw=False)
res.equals(res_parallel)

CPU times: user 2.11 s, sys: 139 ms, total: 2.24 s
Wall time: 26.3 s


# Series.map

In [15]:
df_size = int(5e7)
df = pd.DataFrame(dict(a=np.random.rand(df_size) + 1))
def func(x):
    return math.log10(math.sqrt(math.exp(x**2)))

In [16]:
%%time
res = df.a.map(func)

CPU times: user 22.3 s, sys: 2.3 s, total: 24.6 s
Wall time: 25.8 s


In [17]:
%%time
res_parallel = df.a.parallel_map(func)
res.equals(res_parallel)





CPU times: user 2.08 s, sys: 1.11 s, total: 3.19 s
Wall time: 26.6 s


# Series.apply

In [18]:
df_size = int(3.5e7)
df = pd.DataFrame(dict(a=np.random.rand(df_size) + 1))
def func(x, power, bias=0):
    return math.log10(math.sqrt(math.exp(x**power))) + bias

In [19]:
%%time
res = df.a.apply(func, args=(2,), bias=3)

CPU times: user 23.5 s, sys: 1.16 s, total: 24.6 s
Wall time: 25.3 s


In [20]:
%%time
res_parallel = df.a.parallel_apply(func, args=(2,), bias=3)
res.equals(res_parallel)





CPU times: user 1.58 s, sys: 777 ms, total: 2.36 s
Wall time: 22.8 s


# Series.rolling.apply

In [21]:
df_size = int(1e6)
df = pd.DataFrame(dict(a=np.random.randint(1, 8, df_size),
b=list(range(df_size))))
def func(x):
    return x.iloc[0] + x.iloc[1] ** 2 + x.iloc[2] ** 3 + x.iloc[3] ** 4

In [22]:
%%time
res = df.b.rolling(4).apply(func, raw=False)

CPU times: user 1min 12s, sys: 227 ms, total: 1min 12s
Wall time: 1min 12s


In [23]:
%%time
res_parallel = df.b.rolling(4).parallel_apply(func, raw=False)
res.equals(res_parallel)





CPU times: user 417 ms, sys: 223 ms, total: 639 ms
Wall time: 23.8 s
